In [10]:
import numpy as np
import gdal
import ogr
from skimage import exposure
from skimage.segmentation import quickshift
from skimage.segmentation import slic
import time
import scipy
#import geopandas as gpd
import pandas as pd



In [23]:
#Load Tiff
naip_fn = 'data/S3E105.tif'

In [24]:
#set driver for geotiff
driverTiff = gdal.GetDriverByName('GTiff')
#open geotiff with gdal
naip_ds = gdal.Open(naip_fn)
#count the number of color bands in geotiff
nbands = naip_ds.RasterCount
#Create empty list
band_data = []
print('bands', naip_ds.RasterCount, 'rows', naip_ds.RasterYSize, 'columns', naip_ds.RasterXSize)

#loop through the bands in nbands and append to empty band_data list
for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)
#stack arrays
band_data = np.dstack(band_data)
print(band_data.shape)



bands 4 rows 10560 columns 10560
(10560, 10560, 4)


In [25]:
#rescale band data
img = exposure.rescale_intensity(band_data)

In [26]:
#create segments
segments = slic(img, n_segments=500000, compactness=0.1)

In [27]:
#Save segments to raster
segments_fn = 'data/segments.tif'
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None